#### BI1 Mini-Project: UOL_Bibliography

In [1]:
import pandas as pd
import os
import glob
import numpy as np
import re
  

In [2]:
path = os.getcwd()
csv_files = glob.glob(os.path.join(path, "Data/uol-bibliography-data-2008-2017-raw/", "*.csv"))

In [3]:
df = pd.core.frame.DataFrame(
    {
        "Fach":[],
        "Autor/in":[],
        "Titel":[],
        "Typ":[],
        "Meldetag":[],
        "Punktzahl":[],
        "ZahlOldenburgerAutoren":[],
        "Jahr":[],
        "Faculty":[]
    })
print(df)

Empty DataFrame
Columns: [Fach, Autor/in, Titel, Typ, Meldetag, Punktzahl, ZahlOldenburgerAutoren, Jahr, Faculty]
Index: []


In [4]:
for f in csv_files:
    tmp = pd.read_csv(f)
    try:
        tmp = tmp.assign(Faculty = len(tmp)*re.findall('Fakultt ([1-9])',f))
    except:
        tmp = tmp.assign(Faculty = len(tmp)*[np.nan])
    df = pd.concat([df,tmp],ignore_index=True)
print(df)

                                                    Fach            Autor/in  \
0      Fakultät 5 - Mathematik und Naturwissenschafte...  Ahrenholtz, Ingrid   
1      Fakultät 5 - Mathematik und Naturwissenschafte...         Ruf, Andrea   
2                                   Rechtswissenschaften      Balitzki, Anja   
3                                   Rechtswissenschaften      Balitzki, Anja   
4                                   Rechtswissenschaften      Balitzki, Anja   
...                                                  ...                 ...   
56871                                     Stadtforschung      Siebel, Walter   
56872                                     Stadtforschung      Siebel, Walter   
56873                                     Stadtforschung      Siebel, Walter   
56874                                     Stadtforschung       Wehrheim, Jan   
56875                                     Stadtforschung       Wehrheim, Jan   

                                       